## Assignment 2: R
- SALAMANCA FERNANDEZ, LUCAS PABLO
- PADILLA AQUISE, ALESSANDRO PIERO
- RIEGA NUÑEZ, GABRIEL ANTONIO FERMIN
- SILVA ANDUJAR, NICOLAS
- GARCIA RODRIGUEZ, EMILIO ALONSO


## 1. Loading and processing the data

In [5]:
  install.packages("xtable")
  install.packages("hdm")
  install.packages("glmnet")
  install.packages("ggplot2")
  install.packages("tidyr")
  install.packages("psych")


Installing package into ‘/usr/local/lib/R/site-library’
(as ‘lib’ is unspecified)

Installing package into ‘/usr/local/lib/R/site-library’
(as ‘lib’ is unspecified)

also installing the dependencies ‘iterators’, ‘foreach’, ‘shape’, ‘RcppEigen’, ‘glmnet’, ‘checkmate’, ‘Formula’


Installing package into ‘/usr/local/lib/R/site-library’
(as ‘lib’ is unspecified)

Installing package into ‘/usr/local/lib/R/site-library’
(as ‘lib’ is unspecified)

Installing package into ‘/usr/local/lib/R/site-library’
(as ‘lib’ is unspecified)

Installing package into ‘/usr/local/lib/R/site-library’
(as ‘lib’ is unspecified)

also installing the dependencies ‘mnormt’, ‘GPArotation’




In [6]:

  library(readr)
  library(dplyr)
  library(tidyr)
  library(psych)
  library(ggplot2)
  library(hdm)
  library(glmnet)
  library(xtable)
  library(tidyr)


Attaching package: ‘dplyr’


The following objects are masked from ‘package:stats’:

    filter, lag


The following objects are masked from ‘package:base’:

    intersect, setdiff, setequal, union



Attaching package: ‘ggplot2’


The following objects are masked from ‘package:psych’:

    %+%, alpha


Loading required package: Matrix


Attaching package: ‘Matrix’


The following objects are masked from ‘package:tidyr’:

    expand, pack, unpack


Loaded glmnet 4.1-8



1. Load the data

In [7]:
path <- "/wage2015_subsample_inference.csv"
dt <- read_csv(path)
head(dt)
colnames(dt)

Rows: 5150 Columns: 21
── Column specification ────────────────────────────────────────────────────────
Delimiter: ","
dbl (21): rownames, wage, lwage, sex, shs, hsg, scl, clg, ad, mw, so, we, ne...

ℹ Use `spec()` to retrieve the full column specification for this data.
ℹ Specify the column types or set `show_col_types = FALSE` to quiet this message.


rownames,wage,lwage,sex,shs,hsg,scl,clg,ad,mw,⋯,we,ne,exp1,exp2,exp3,exp4,occ,occ2,ind,ind2
<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,⋯,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
10,9.615385,2.263364,1,0,0,0,1,0,0,⋯,0,1,7,0.49,0.343,0.2401,3600,11,8370,18
12,48.076923,3.872802,0,0,0,0,1,0,0,⋯,0,1,31,9.61,29.791,92.3521,3050,10,5070,9
15,11.057692,2.403126,0,0,1,0,0,0,0,⋯,0,1,18,3.24,5.832,10.4976,6260,19,770,4
18,13.942308,2.634928,1,0,0,0,0,1,0,⋯,0,1,25,6.25,15.625,39.0625,420,1,6990,12
19,28.846154,3.361977,1,0,0,0,1,0,0,⋯,0,1,22,4.84,10.648,23.4256,2015,6,9470,22
30,11.730769,2.462215,1,0,0,0,1,0,0,⋯,0,1,1,0.01,0.001,0.0001,1650,5,7460,14


[1] "rownames" "wage"     "lwage"    "sex"      "shs"      "hsg"     
 [7] "scl"      "clg"      "ad"       "mw"       "so"       "we"      
[13] "ne"       "exp1"     "exp2"     "exp3"     "exp4"     "occ"     
[19] "occ2"     "ind"      "ind2"

2 Generate the extra-flexible model

In [8]:
var_extraflex <- dt %>%
  select(sex, exp1, exp2, exp3, exp4, hsg, scl, clg, ad, so, we, ne, starts_with("occ2_"), starts_with("ind2_"))

In [9]:
var_names <- names(var_extraflex)
print(var_names)

 [1] "sex"  "exp1" "exp2" "exp3" "exp4" "hsg"  "scl"  "clg"  "ad"   "so"  
[11] "we"   "ne"  


In [10]:
#Make the interactions
interaction_combinations <- expand.grid(var_names, var_names, stringsAsFactors = FALSE)
interaction_combinations <- interaction_combinations[interaction_combinations$Var1 != interaction_combinations$Var2, ]

In [11]:
for (i in 1:nrow(interaction_combinations)) {
  var1 <- interaction_combinations$Var1[i]
  var2 <- interaction_combinations$Var2[i]
  if (var1 %in% names(var_extraflex) & var2 %in% names(var_extraflex)) {
    interaction_name <- paste(var1, var2, "interaction", sep = "_")
    var_extraflex[[interaction_name]] <- var_extraflex[[var1]] * var_extraflex[[var2]]
  }
}
var_extraflex <- var_extraflex %>% distinct()
extraflex_array <- as.matrix(var_extraflex)

In [12]:
print("Array model extra-flexible:")
print(extraflex_array)

Se han truncado las últimas 5000 líneas del flujo de salida.
 [561,]                 0                 0                  0
 [562,]                 0                 0                  0
 [563,]                 0                 0                  0
 [564,]                 0                 0                  0
 [565,]                 0                 0                  0
 [566,]                 0                 0                  0
 [567,]                 0                 0                  0
 [568,]                 0                 0                  0
 [569,]                 0                 0                  0
 [570,]                 0                 0                  0
 [571,]                 0                 0                  0
 [572,]                 0                 0                  0
 [573,]                 0                 0                  0
 [574,]                 0                 0                  0
 [575,]                 0                 0              

In [13]:
#Normalization
dt$lwage_tilde <- (dt$lwage - mean(dt$lwage, na.rm = TRUE)) / sd(dt$lwage, na.rm = TRUE)
numeric_vars <- c("wage", "exp1", "exp2", "exp3", "exp4", "lwage_tilde")
for (var in numeric_vars) {
  dt[[paste0(var, "_tilde")]] <- (dt[[var]] - mean(dt[[var]], na.rm = TRUE)) / sd(dt[[var]], na.rm = TRUE)
}

In [14]:
#Normalize Variable Y
Y <- (dt$wage - mean(dt$wage, na.rm = TRUE)) / sd(dt$wage, na.rm = TRUE)
Y_array <- matrix(Y, ncol = 1)
print("Array of variable Y :")
print(Y_array)

Se han truncado las últimas 5000 líneas del flujo de salida.
 [151,] -0.305213785
 [152,] -0.588140182
 [153,] -0.656811638
 [154,] -0.633921153
 [155,] -0.588140182
 [156,] -0.313454359
 [157,] -0.067152738
 [158,] -0.382125815
 [159,]  0.029902920
 [160,]  0.602165051
 [161,]  0.178668183
 [162,] -0.199001933
 [163,] -0.272251486
 [164,] -0.043346633
 [165,] -0.097093493
 [166,] -0.199001933
 [167,] -0.405016300
 [168,] -0.162377157
 [169,] -0.627511817
 [170,]  1.174427182
 [171,] -0.402472913
 [172,] -0.427906786
 [173,]  0.004469047
 [174,]  0.207940027
 [175,]  0.002434337
 [176,]  0.515181207
 [177,]  0.533493595
 [178,] -0.679702123
 [179,] -0.313454359
 [180,]  0.716617477
 [181,] -0.313454359
 [182,] -0.313454359
 [183,] -0.530183422
 [184,] -0.638499250
 [185,] -0.496578241
 [186,] -0.322258392
 [187,] -0.427906786
 [188,]  0.069369273
 [189,] -0.698430702
 [190,] -0.097266443
 [191,]  0.011590531
 [192,] -0.542359212
 [193,] -0.580510021
 [194,] -0.656811638
 [195,] -0.7859

2.2. Generate the array for the predictors X (do not generate an intercept) and normalize its colums.

In [15]:
#Eliminate intercept
predictor_vars <- var_extraflex %>% select(-sex)
X_array <- as.matrix(predictor_vars)

In [16]:
#Normalize X
X_array_normalized <- apply(X_array, 2, function(col) (col - mean(col, na.rm = TRUE)) / sd(col, na.rm = TRUE)) #FORM 1

In [17]:
print("Array de predictores X:")
print(X_array)

Se han truncado las últimas 5000 líneas del flujo de salida.
 [601,]                 0                 0                  0
 [602,]                 0                 0                  0
 [603,]                 0                 0                  0
 [604,]                 0                 0                  0
 [605,]                 0                 0                  0
 [606,]                 0                 0                  0
 [607,]                 0                 0                  0
 [608,]                 0                 0                  0
 [609,]                 0                 0                  0
 [610,]                 0                 0                  0
 [611,]                 0                 0                  0
 [612,]                 0                 0                  0
 [613,]                 0                 0                  0
 [614,]                 0                 0                  0
 [615,]                 0                 0              

In [18]:
X_array_normalized <- scale(X_array) #FORM 2
print("Array de predictores X:")
print(X_array)

Se han truncado las últimas 5000 líneas del flujo de salida.
 [601,]                 0                 0                  0
 [602,]                 0                 0                  0
 [603,]                 0                 0                  0
 [604,]                 0                 0                  0
 [605,]                 0                 0                  0
 [606,]                 0                 0                  0
 [607,]                 0                 0                  0
 [608,]                 0                 0                  0
 [609,]                 0                 0                  0
 [610,]                 0                 0                  0
 [611,]                 0                 0                  0
 [612,]                 0                 0                  0
 [613,]                 0                 0                  0
 [614,]                 0                 0                  0
 [615,]                 0                 0              

3. Split between training and testing samples. The testing sample should be 10% of the total.

In [19]:
set.seed(123)
train_indices <- sample(1:nrow(dt), size = 0.9 * nrow(dt)) #Establecido al 10% (0.9)

In [20]:
#Extra-flexible Model
extra_flexible_model_data <- dt[, c("lwage_tilde", "sex", "exp1_tilde", "exp2_tilde", "exp3_tilde", "exp4_tilde", "hsg", "scl", "clg", "ad", "so", "we", "ne", "occ2", "ind2")]

In [21]:
#Split the data withing the train and the test
extra_flexible_train <- extra_flexible_model_data[train_indices, ]
extra_flexible_test <- extra_flexible_model_data[-train_indices, ]

In [22]:
cat("Training sample size (extra-flexible model):", nrow(extra_flexible_train), "\n")
cat("Test sample size (extra-flexible model):", nrow(extra_flexible_test), "\n")

Training sample size (extra-flexible model): 4635 
Test sample size (extra-flexible model): 515 


In [23]:
## Creating the Lasso Cross-Validation Procedure

## 2. Creating the Lasso Cross-Validation Procedure

4. Program a function that generates a logarithmically spaced grid. The input arguments should be the natural logarithms of the lower and upper bounds of the grid, as well as the natural logarithm of the spacing between each element of the grid. The output should be the logarithmically spaced grid, meaning that if we take the natural logarithm of each entry in the grid, they will be equally spaced. This will be the grid of values for λ values to try during cross-validation.

In [24]:
log_spaced_grid <- function(ln_lower, ln_upper, ln_spacing) {
  lower <- exp(ln_lower)
  upper <- exp(ln_upper)

  num_points <- ceiling((ln_upper - ln_lower) / ln_spacing) + 1

  grid <- exp(seq(ln_lower, ln_upper, length.out = num_points))

  return(grid)
}

In [25]:
#TEST
ln_lower <- log(1)
ln_upper <- log(100)
ln_spacing <- log(2)
lambda_grid <- log_spaced_grid(ln_lower, ln_upper, ln_spacing)
print(lambda_grid)

[1]   1.000000   1.930698   3.727594   7.196857  13.894955  26.826958  51.794747
[8] 100.000000


5. Program a function to generate k folds. It should take as input the array to be split rowwise and the number of folds desired. It should output a list of k 1d arrays of booleans; these arrays should all be the same length as the number of rows in the input array, and when they are all summed together they should add up to an array of all true values. Create your own procedure for splitting. You can aid yourself with third party packages like numpy in Python or Statistics in Julia, but do not use a pre-programmed third party splitting procedure like sk-learns's KFolds in Python

In [26]:
# forma 1
# Función para generar k pliegues
create_k_folds <- function(data, k) {
  # Número de filas en el conjunto de datos
  n <- nrow(data)

  # Crear un vector de índices
  indices <- sample(1:n)

  # Dividir los índices en k pliegues
  folds <- split(indices, cut(1:n, breaks = k, labels = FALSE))

  # Crear una lista de arreglos booleanos
  folds_list <- vector("list", k)
  for (i in 1:k) {
    # Inicializar un vector lógico con FALSE
    fold <- rep(FALSE, n)
    # Establecer TRUE para las filas en el pliegue actual
    fold[folds[[i]]] <- TRUE
    folds_list[[i]] <- fold
  }

  return(folds_list)
}

# Prueba de la función
set.seed(123) # Para reproducibilidad
k <- 5
folds <- create_k_folds(dt, k)

# Mostrar el tamaño de cada pliegue
lapply(folds, sum)

# Mostrar los resultados de cada pliegue
for (i in 1:number_of_folds) {
  cat("Fold", i, ":", paste(which(k_folds_result[[i]]), collapse = ", "), "\n")
}



[[1]]
[1] 1030

[[2]]
[1] 1030

[[3]]
[1] 1030

[[4]]
[1] 1030

[[5]]
[1] 1030

Fold 1 : 13, 17, 18, 19, 35, 36, 41, 46, 48, 51, 56, 57, 68, 72, 74, 78, 80, 83, 92, 96 
Fold 2 : 1, 3, 11, 28, 31, 33, 38, 39, 42, 43, 50, 54, 61, 66, 69, 70, 71, 84, 89, 99 
Fold 3 : 2, 4, 5, 6, 10, 12, 20, 24, 26, 29, 37, 40, 49, 52, 67, 73, 85, 87, 88, 95 
Fold 4 : 8, 9, 22, 23, 27, 30, 32, 34, 44, 45, 53, 55, 63, 75, 79, 82, 86, 91, 94, 100 
Fold 5 : 7, 14, 15, 16, 21, 25, 47, 58, 59, 60, 62, 64, 65, 76, 77, 81, 90, 93, 97, 98 


In [27]:
# forma 2
generate_k_folds <- function(data, k) {
  total_rows <- nrow(data)
  randomized_indices <- sample(1:total_rows)
  folds <- vector("list", k)

  # Inicializar los pliegues como vectores lógicos
  for (i in 1:k) {
    folds[[i]] <- rep(FALSE, total_rows)
  }

  # Distribuir aleatoriamente las filas en los pliegues
  for (index in 1:total_rows) {
    current_fold <- (index - 1) %% k + 1
    folds[[current_fold]][randomized_indices[index]] <- TRUE
  }

  return(folds)
}

# TEST
set.seed(456)
example_data <- matrix(1:100, nrow = 100, ncol = 1)
number_of_folds <- 5
k_folds_result <- generate_k_folds(example_data, number_of_folds)

# Mostrar los resultados de cada pliegue
for (i in 1:number_of_folds) {
  cat("Fold", i, ":", which(k_folds_result[[i]]), "\n")
}


Fold 1 : 13 17 18 19 35 36 41 46 48 51 56 57 68 72 74 78 80 83 92 96 
Fold 2 : 1 3 11 28 31 33 38 39 42 43 50 54 61 66 69 70 71 84 89 99 
Fold 3 : 2 4 5 6 10 12 20 24 26 29 37 40 49 52 67 73 85 87 88 95 
Fold 4 : 8 9 22 23 27 30 32 34 44 45 53 55 63 75 79 82 86 91 94 100 
Fold 5 : 7 14 15 16 21 25 47 58 59 60 62 64 65 76 77 81 90 93 97 98 


6.Program a function that integrates those that you programmed in the last two items to find the value of
λ
 that minimizes the testing mean square error across folds. It should take the following inputs:

Y: an array for the outcome variable.
X: an array of predictors.
lambda_bounds: the lower and upper bounds of the grid of lambda values.
k: number of folds
The output should be a dictionary (a list in R) with the following entries:

optimal_lambda: The lambda that minimizes the testing MSE across folds.
optimal_coef: An array with the coefficients found for the optimal lambda
all_lambdas: The grid of lambdas.
all_mse: An array with the testing MSE across folds for each lambda.
The procedure goes as follows:

With each lambda in the grid
For each split
Train and test a lasso model.
With this, you can get the testing MSE associated with the value of lambda in each iteration. Feel free to use a third party Lasso estimator, as long as it is not one that calculates the optimal lambda value. Instead, the third party estimator you use should only estimate the regression with the penalty weight provided so you can get the testing MSE with that specific weight.

In [35]:
Y <- dt$log_wage


Warning message:
“Unknown or uninitialised column: `log_wage`.”


In [36]:
X <- as.matrix(dt[, -which(names(dt) == "log_wage")])


In [37]:
# Verificar los nombres de las columnas
colnames(dt)

# Asignar la variable dependiente (ajusta el nombre de la columna si es necesario)
Y <- dt$log_wage  # Asegúrate de usar el nombre correcto

# Asignar las variables predictoras (X) eliminando la columna dependiente
X <- as.matrix(dt[, -which(names(dt) == "log_wage")])

# Crear el grid de lambdas
lambda_bounds <- crear_grid_lambdas(0.01, 10, 0.1)

# Establecer el número de folds
k <- 5  # número de folds

# Ejecutar la validación cruzada Lasso
results <- lasso_cross_validation(Y, X, lambda_bounds, k)


[1] "rownames"          "wage"              "lwage"            
 [4] "sex"               "shs"               "hsg"              
 [7] "scl"               "clg"               "ad"               
[10] "mw"                "so"                "we"               
[13] "ne"                "exp1"              "exp2"             
[16] "exp3"              "exp4"              "occ"              
[19] "occ2"              "ind"               "ind2"             
[22] "lwage_tilde"       "wage_tilde"        "exp1_tilde"       
[25] "exp2_tilde"        "exp3_tilde"        "exp4_tilde"       
[28] "lwage_tilde_tilde"

Warning message:
“Unknown or uninitialised column: `log_wage`.”


Dimensiones iniciales de X: 5150 0 
Longitud inicial de Y: 0 


ERROR: Error in lasso_cross_validation(Y, X, lambda_bounds, k): No hay datos válidos después del filtrado.


In [38]:
# Asignar la variable dependiente
Y <- dt$lwage  # Usar el nombre correcto de la columna

# Asignar las variables predictoras eliminando 'lwage'
X <- as.matrix(dt[, -which(names(dt) == "lwage")])

# Crear el grid de lambdas
lambda_bounds <- crear_grid_lambdas(0.01, 10, 0.1)

# Establecer el número de folds
k <- 5  # número de folds

# Ejecutar la validación cruzada Lasso
results <- lasso_cross_validation(Y, X, lambda_bounds, k)


Dimensiones iniciales de X: 5150 27 
Longitud inicial de Y: 5150 
Dimensiones de X después del filtrado: 5150 27 
Longitud de Y después del filtrado: 5150 
Mejor lambda: 0.01 


In [39]:
# Ajustar el modelo Lasso final con el mejor lambda encontrado
final_lasso_model <- glmnet(X, Y, alpha = 1, lambda = results$best_lambda)

# Obtener los coeficientes del modelo ajustado
coeficientes <- coef(final_lasso_model, s = results$best_lambda)

# Mostrar los coeficientes
print(coeficientes)


28 x 1 sparse Matrix of class "dgCMatrix"
                            s1
(Intercept)       2.970787e+00
rownames          .           
wage              .           
sex               .           
shs               .           
hsg               .           
scl               .           
clg               .           
ad                .           
mw                .           
so                .           
we                .           
ne                .           
exp1              .           
exp2              .           
exp3              .           
exp4              .           
occ               .           
occ2              .           
ind               .           
ind2              .           
lwage_tilde       5.603838e-01
wage_tilde        .           
exp1_tilde        .           
exp2_tilde        .           
exp3_tilde        .           
exp4_tilde        .           
lwage_tilde_tilde 1.717504e-14


7 Program a function for predicting the outcome variable through model estimated with the optimal lambda. It should take as inputs

optimal_model: A dictionary with the values outputed by the function defined for the previous point.
X: an array of predictors.
The output should be an array of predicted values.

In [41]:
install.packages("caret")


Installing package into ‘/usr/local/lib/R/site-library’
(as ‘lib’ is unspecified)

also installing the dependencies ‘listenv’, ‘parallelly’, ‘future’, ‘globals’, ‘future.apply’, ‘numDeriv’, ‘progressr’, ‘SQUAREM’, ‘diagram’, ‘lava’, ‘prodlim’, ‘proxy’, ‘clock’, ‘gower’, ‘hardhat’, ‘ipred’, ‘timeDate’, ‘e1071’, ‘ModelMetrics’, ‘plyr’, ‘pROC’, ‘recipes’, ‘reshape2’




In [42]:
library(caret)


Loading required package: lattice



In [50]:
# Suponiendo que `outcome_train` es un vector y `data_train` es un dataframe
X_train <- as.matrix(data_train)  # Convertir las variables independientes a matriz
y_train <- outcome_train            # Definir la variable dependiente


In [51]:
# Ajustar el modelo Lasso
lasso_model <- glmnet(X_train, y_train, alpha = 1)

# Encontrar el mejor lambda usando validación cruzada
cv_model <- cv.glmnet(X_train, y_train, alpha = 1)

# El mejor lambda
optimal_lambda <- cv_model$lambda.min
print(optimal_lambda)

# Coeficientes del modelo Lasso
coef(lasso_model, s = optimal_lambda)


[1] 0.6321437


22 x 1 sparse Matrix of class "dgCMatrix"
                   s1
(Intercept) 0.6859496
rownames    .        
wage        0.9708495
lwage       .        
sex         .        
shs         .        
hsg         .        
scl         .        
clg         .        
ad          .        
mw          .        
so          .        
we          .        
ne          .        
exp1        .        
exp2        .        
exp3        .        
exp4        .        
occ         .        
occ2        .        
ind         .        
ind2        .        

In [52]:
# Suponiendo que `data_test` es tu conjunto de prueba
X_test <- as.matrix(data_test)  # Convertir las variables independientes a matriz
lasso_predictions <- predict(lasso_model, newx = X_test, s = optimal_lambda)


In [53]:

library(glmnet)

# Preparar los datos
X_train <- as.matrix(data_train)  # Convertir las variables independientes a matriz
y_train <- outcome_train            # Definir la variable dependiente

# Ajustar el modelo Lasso
lasso_model <- glmnet(X_train, y_train, alpha = 1)

# Validación cruzada para encontrar el mejor lambda
cv_model <- cv.glmnet(X_train, y_train, alpha = 1)
optimal_lambda <- cv_model$lambda.min

# Imprimir el mejor lambda
print(paste("Optimal lambda:", optimal_lambda))

# Coeficientes del modelo Lasso
lasso_coefficients <- coef(lasso_model, s = optimal_lambda)
print("Coefficients:")
print(lasso_coefficients)

# Predicciones en el conjunto de prueba
X_test <- as.matrix(data_test)  # Convertir las variables independientes a matriz
lasso_predictions <- predict(lasso_model, newx = X_test, s = optimal_lambda)

# Resultado de las predicciones (puedes ajustar según necesites)
print("Predictions for the test set:")
print(lasso_predictions)


[1] "Optimal lambda: 0.632143702771895"
[1] "Coefficients:"
22 x 1 sparse Matrix of class "dgCMatrix"
                   s1
(Intercept) 0.6859496
rownames    .        
wage        0.9708495
lwage       .        
sex         .        
shs         .        
hsg         .        
scl         .        
clg         .        
ad          .        
mw          .        
so          .        
we          .        
ne          .        
exp1        .        
exp2        .        
exp3        .        
exp4        .        
occ         .        
occ2        .        
ind         .        
ind2        .        
[1] "Predictions for the test set:"
            s1
4    14.221832
6    12.074761
15   16.555604
33   10.487795
43    7.220513
48    7.687268
62   19.356132
73   10.954550
74   10.954550
80   10.021041
85   21.689904
86    8.983808
91   19.356132
93   12.354813
106  14.221832
110  18.289264
111  11.183926
115  15.207202
126  14.688586
155  11.421304
157  22.044638
174  27.653990
176  33.918


Applying the Lasso Cross-Validation Procedure 8 Fit a simple OLS model with the training sample.





8 Fit a simple OLS model with the training sample.




In [54]:
# Ajustar un modelo OLS con la muestra de entrenamiento
ols_model <- lm(outcome_train ~ ., data = data_train)

# Resumen del modelo OLS
summary(ols_model)


Warning message in summary.lm(ols_model):
“essentially perfect fit: summary may be unreliable”



Call:
lm(formula = outcome_train ~ ., data = data_train)

Residuals:
       Min         1Q     Median         3Q        Max 
-5.050e-13 -2.800e-16  1.100e-16  5.400e-16  3.437e-14 

Coefficients: (2 not defined because of singularities)
              Estimate Std. Error    t value Pr(>|t|)    
(Intercept) -5.962e-15  1.198e-15 -4.976e+00 6.73e-07 ***
rownames    -4.564e-20  4.472e-20 -1.021e+00    0.307    
wage         1.000e+00  8.095e-18  1.235e+17  < 2e-16 ***
lwage        2.810e-16  3.336e-16  8.420e-01    0.400    
sex         -1.104e-16  2.338e-16 -4.720e-01    0.637    
shs          5.262e-16  8.387e-16  6.270e-01    0.530    
hsg          8.883e-17  4.448e-16  2.000e-01    0.842    
scl         -3.018e-16  4.081e-16 -7.400e-01    0.460    
clg         -7.818e-17  3.650e-16 -2.140e-01    0.830    
ad                  NA         NA         NA       NA    
mw          -4.081e-17  4.569e-16 -8.900e-02    0.929    
so           7.248e-16  7.990e-16  9.070e-01    0.364    
we      

9
Find the optimal lambda and its related coefficients with the function programmed in 3. and the training sample. The natural logarithm of the numbers in the grid of
λ
 s should be spaced by 0.1, and the natural logarithm of the lower and upper bounds of the grid should be -7 and 7 respectively. For the k-folds procedure, use 5 folds. Print the lambda and the coefficients.

In [75]:
# Verifica el número de variables en el conjunto de datos de prueba
ncol(data_test)


[1] 21

In [77]:
# Verifica las dimensiones del conjunto de datos de prueba
dim(data_test)


[1] 515  21

In [79]:
# Verifica las dimensiones de data_test
dim(data_test)

# Verifica los nombres de las columnas de data_test
colnames(data_test)

# Verifica los coeficientes del modelo lasso_model
coef(lasso_model)


[1] 515  21

[1] "rownames" "wage"     "lwage"    "sex"      "shs"      "hsg"     
 [7] "scl"      "clg"      "ad"       "mw"       "so"       "we"      
[13] "ne"       "exp1"     "exp2"     "exp3"     "exp4"     "occ"     
[19] "occ2"     "ind"      "ind2"

  [[ suppressing 39 column names ‘s0’, ‘s1’, ‘s2’ ... ]]



22 x 39 sparse Matrix of class "dgCMatrix"
                                                                            
(Intercept) 23.53129 21.44083408 19.5360895 17.8005571 16.2192047 14.7783352
rownames     .        .           .          .          .          .        
wage         .        0.08883724  0.1697824  0.2435367  0.3107388  0.3719709
lwage        .        .           .          .          .          .        
sex          .        .           .          .          .          .        
shs          .        .           .          .          .          .        
hsg          .        .           .          .          .          .        
scl          .        .           .          .          .          .        
clg          .        .           .          .          .          .        
ad           .        .           .          .          .          .        
mw           .        .           .          .          .          .        
so           .        .          

In [80]:
# Selecciona las columnas necesarias en data_test
data_test_filtered <- data_test[, c("wage", "lwage", "sex", "shs", "hsg", "scl", "clg",
                                     "ad", "mw", "so", "we", "ne", "exp1", "exp2",
                                     "exp3", "exp4", "occ", "occ2", "ind", "ind2")]

# Verifica las dimensiones del nuevo conjunto de datos de prueba
dim(data_test_filtered)


[1] 515  20

In [81]:
# Agregar una columna de intercepto a data_test_filtered
data_test_final <- cbind(Intercept = 1, data_test_filtered)

# Verifica las dimensiones del nuevo conjunto de datos de prueba
dim(data_test_final)


[1] 515  21

In [82]:
# Realizar predicciones en el conjunto de prueba
predictions <- predict(lasso_model, newx = as.matrix(data_test_final))

# Verifica los primeros valores de las predicciones
head(predictions)


,s0,s1,s2,s3,s4,s5,s6,s7,s8,s9,⋯,s29,s30,s31,s32,s33,s34,s35,s36,s37,s38
4,23.53129,22.67943,21.90325,21.19602,20.55162,19.96447,19.42948,18.94201,18.49785,18.09315,⋯,14.588044,14.530679,14.478409,14.430784,14.387389,14.347849,14.311822,14.278995,14.249085,14.221832
6,23.53129,22.48296,21.52777,20.65743,19.86441,19.14184,18.48346,17.88357,17.33697,16.83893,⋯,12.525434,12.454838,12.390514,12.331904,12.278501,12.229842,12.185506,12.145108,12.108299,12.074761
15,23.53129,22.89298,22.31138,21.78145,21.29859,20.85863,20.45775,20.09249,19.75968,19.45643,⋯,16.830012,16.787027,16.747861,16.712175,16.679658,16.650031,16.623035,16.598438,16.576026,16.555604
33,23.53129,22.33775,21.25024,20.25934,19.35647,18.53381,17.78423,17.10124,16.47893,15.91190,⋯,11.000896,10.920521,10.847287,10.780558,10.719758,10.664358,10.613881,10.567887,10.525980,10.487795
43,23.53129,22.03878,20.67886,19.43975,18.31072,17.28199,16.34464,15.49057,14.71238,14.00331,⋯,7.862142,7.761634,7.670054,7.586611,7.510580,7.441304,7.378182,7.320668,7.268263,7.220513
48,23.53129,22.08149,20.76048,19.55683,18.46011,17.46082,16.55030,15.72067,14.96474,14.27597,⋯,8.310535,8.212903,8.123945,8.042889,7.969034,7.901740,7.840425,7.784556,7.733651,7.687268


In [83]:
# Extraer los valores reales del conjunto de prueba
actual_values <- data_test_final$outcome

# Calcular el error cuadrático medio
mse <- mean((predictions - actual_values)^2)

# Mostrar el MSE
mse


[1] NaN

In [84]:
# Verificar NA en las predicciones
sum(is.na(predictions))

# Verificar NA en los valores reales
sum(is.na(actual_values))


[1] 0

[1] 0

In [86]:
length(predictions)
length(actual_values)


[1] 20085

[1] 0

In [87]:
str(predictions)
str(actual_values)


 num [1:515, 1:39] 23.5 23.5 23.5 23.5 23.5 ...
 - attr(*, "dimnames")=List of 2
  ..$ : chr [1:515] "4" "6" "15" "33" ...
  ..$ : chr [1:39] "s0" "s1" "s2" "s3" ...
 NULL


In [88]:
rmse <- sqrt(mean((predictions - actual_values)^2))


In [89]:
head(predictions)
head(actual_values)


,s0,s1,s2,s3,s4,s5,s6,s7,s8,s9,⋯,s29,s30,s31,s32,s33,s34,s35,s36,s37,s38
4,23.53129,22.67943,21.90325,21.19602,20.55162,19.96447,19.42948,18.94201,18.49785,18.09315,⋯,14.588044,14.530679,14.478409,14.430784,14.387389,14.347849,14.311822,14.278995,14.249085,14.221832
6,23.53129,22.48296,21.52777,20.65743,19.86441,19.14184,18.48346,17.88357,17.33697,16.83893,⋯,12.525434,12.454838,12.390514,12.331904,12.278501,12.229842,12.185506,12.145108,12.108299,12.074761
15,23.53129,22.89298,22.31138,21.78145,21.29859,20.85863,20.45775,20.09249,19.75968,19.45643,⋯,16.830012,16.787027,16.747861,16.712175,16.679658,16.650031,16.623035,16.598438,16.576026,16.555604
33,23.53129,22.33775,21.25024,20.25934,19.35647,18.53381,17.78423,17.10124,16.47893,15.91190,⋯,11.000896,10.920521,10.847287,10.780558,10.719758,10.664358,10.613881,10.567887,10.525980,10.487795
43,23.53129,22.03878,20.67886,19.43975,18.31072,17.28199,16.34464,15.49057,14.71238,14.00331,⋯,7.862142,7.761634,7.670054,7.586611,7.510580,7.441304,7.378182,7.320668,7.268263,7.220513
48,23.53129,22.08149,20.76048,19.55683,18.46011,17.46082,16.55030,15.72067,14.96474,14.27597,⋯,8.310535,8.212903,8.123945,8.042889,7.969034,7.901740,7.840425,7.784556,7.733651,7.687268


NULL

In [90]:
any(predictions != actual_values)  # Esto debería ser TRUE si hay diferencias


[1] FALSE

In [91]:
rmse <- sqrt(mean((predictions - actual_values)^2))


In [92]:
summary(predictions)
summary(actual_values)


       s0              s1              s2              s3       
 Min.   :23.53   Min.   :21.71   Min.   :20.06   Min.   :18.55  
 1st Qu.:23.53   1st Qu.:22.59   1st Qu.:21.73   1st Qu.:20.95  
 Median :23.53   Median :23.15   Median :22.80   Median :22.48  
 Mean   :23.53   Mean   :23.42   Mean   :23.33   Mean   :23.24  
 3rd Qu.:23.53   3rd Qu.:23.97   3rd Qu.:24.37   3rd Qu.:24.73  
 Max.   :23.53   Max.   :30.15   Max.   :36.19   Max.   :41.69  
       s4              s5              s6              s7       
 Min.   :17.18   Min.   :15.92   Min.   :14.78   Min.   :13.74  
 1st Qu.:20.24   1st Qu.:19.59   1st Qu.:19.00   1st Qu.:18.46  
 Median :22.19   Median :21.93   Median :21.69   Median :21.47  
 Mean   :23.16   Mean   :23.08   Mean   :23.01   Mean   :22.95  
 3rd Qu.:25.06   3rd Qu.:25.37   3rd Qu.:25.64   3rd Qu.:25.89  
 Max.   :46.70   Max.   :51.26   Max.   :55.42   Max.   :59.21  
       s8              s9             s10             s11       
 Min.   :12.79   Min.   :

Length  Class   Mode 
     0   NULL   NULL 

In [93]:
# Imprimir las primeras 10 predicciones y sus valores reales correspondientes
for (i in seq(1, 10)) {
  cat("Predicción:", predictions[i], "Valor real:", actual_values[i], "\n")
}


Predicción: 23.53129 Valor real: 
Predicción: 23.53129 Valor real: 
Predicción: 23.53129 Valor real: 
Predicción: 23.53129 Valor real: 
Predicción: 23.53129 Valor real: 
Predicción: 23.53129 Valor real: 
Predicción: 23.53129 Valor real: 
Predicción: 23.53129 Valor real: 
Predicción: 23.53129 Valor real: 
Predicción: 23.53129 Valor real: 


In [94]:
# Imprimir los valores reales para las primeras 10 observaciones
print(head(actual_values, 10))


NULL


In [97]:
set.seed(123)  # Para reproducibilidad
train_index <- sample(1:nrow(data), 0.9 * nrow(data))
train_data <- data[train_index, ]
test_data <- data[-train_index, ]


In [99]:
actual_values <- test_data$tu_variable_objetivo


In [101]:
# Asegúrate de que tienes un conjunto de prueba
set.seed(123)
train_index <- sample(1:nrow(data), 0.9 * nrow(data))
train_data <- data[train_index, ]
test_data <- data[-train_index, ]

# Ajustar un modelo de regresión (ajusta según tu caso)
model <- lm(wage ~ ., data = train_data)  # Asegúrate de que 'wage' sea la variable objetivo

# Hacer predicciones
predictions <- predict(model, newdata = test_data)

# Define los valores reales
actual_values <- test_data$wage  # Reemplaza 'wage' con tu variable objetivo real

# Imprimir las primeras 10 predicciones y sus valores reales
for (i in seq(1, 10)) {
  cat("Predicción:", predictions[i], "Valor real:", actual_values[i], "\n")
}

# Imprimir los valores reales para verificar
cat("Valores reales (primeras 10):", "\n")
print(head(actual_values, 10))



Predicción: 10.69347 Valor real: 13.94231 
Predicción: 5.996952 Valor real: 11.73077 
Predicción: 15.3017 Valor real: 16.34615 
Predicción: 4.096782 Valor real: 10.09615 
Predicción: -7.403572 Valor real: 6.730769 
Predicción: -6.062286 Valor real: 7.211538 
Predicción: 24.21869 Valor real: 19.23077 
Predicción: 6.74276 Valor real: 10.57692 
Predicción: 5.426881 Valor real: 10.57692 
Predicción: 2.95386 Valor real: 9.615385 
Valores reales (primeras 10): 
 [1] 13.942308 11.730769 16.346154 10.096154  6.730769  7.211538 19.230769
 [8] 10.576923 10.576923  9.615385


10Now use each language's hdm package to fit a Lasso model with the theoretical optimal lambda value. You can find the documentation of the package for each language in the following links


In [102]:
# Asegúrate de que tienes un conjunto de prueba
set.seed(123)
train_index <- sample(1:nrow(data), 0.9 * nrow(data))
train_data <- data[train_index, ]
test_data <- data[-train_index, ]

# Ajustar un modelo de regresión (ajusta según tu caso)
model <- lm(wage ~ ., data = train_data)  # Asegúrate de que 'wage' sea la variable objetivo

# Hacer predicciones
predictions <- predict(model, newdata = test_data)

# Define los valores reales
actual_values <- test_data$wage  # Reemplaza 'wage' con tu variable objetivo real

# Imprimir las primeras 10 predicciones y sus valores reales
for (i in seq(1, 10)) {
  cat("Predicción:", predictions[i], "Valor real:", actual_values[i], "\n")
}

# Calcular el error cuadrático medio (RMSE)
rmse <- sqrt(mean((predictions - actual_values)^2))
cat("Error Cuadrático Medio (RMSE):", rmse, "\n")


Predicción: 10.69347 Valor real: 13.94231 
Predicción: 5.996952 Valor real: 11.73077 
Predicción: 15.3017 Valor real: 16.34615 
Predicción: 4.096782 Valor real: 10.09615 
Predicción: -7.403572 Valor real: 6.730769 
Predicción: -6.062286 Valor real: 7.211538 
Predicción: 24.21869 Valor real: 19.23077 
Predicción: 6.74276 Valor real: 10.57692 
Predicción: 5.426881 Valor real: 10.57692 
Predicción: 2.95386 Valor real: 9.615385 
Error Cuadrático Medio (RMSE): 6.604055 


11 Report the testing MSE and R 2 for the OLS model and, the cross-validation lambda Lasso model, and the hdm theoretical lambda model. Use the function programmed in 4. to compute the predictions of the cross-validation Lasso model.

In [103]:
# Calcular el RMSE para un modelo de referencia (media de los valores)
mean_value <- mean(actual_values)
predictions_ref <- rep(mean_value, length(actual_values))
rmse_ref <- sqrt(mean((predictions_ref - actual_values)^2))

# Comparar los RMSE
cat("RMSE del modelo ajustado:", rmse, "\n")
cat("RMSE del modelo de referencia (media):", rmse_ref, "\n")

# Comentario sobre el rendimiento
if (rmse < rmse_ref) {
  cat("El modelo ajustado es mejor que el modelo de referencia.\n")
} else {
  cat("El modelo ajustado no es mejor que el modelo de referencia.\n")
}


RMSE del modelo ajustado: 6.604055 
RMSE del modelo de referencia (media): 13.29469 
El modelo ajustado es mejor que el modelo de referencia.
